# Limpieza de datos de medicamentos

En este notebook se realiza la limpieza del conjunto de datos.
Se preparan los datos para el análisis eliminando variables no relevantes y verificando la estructura final del DataFrame.


In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('data_medicamentos.csv')
df.head()

,principio_activo,unidad_de_dispensacion,concentracion,unidad_base,nombre_comercial,fabricante,precio_por_tableta,factoresprecio
0,Midazolam,Ampolla,Midazolam 15 mg,ml,Dormicum,Siegfried,11199.800000,Alto
1,Acido Valproico,Tableta,Divalproato Sodico 500 mg,mg,Valcote,Lafrancol,3752.866667,Medio
2,Acido Valproico,Tableta,Divalproato Sodico 500 mg,mg,Valcote,Lafrancol,1777.266522,Medio
3,Fluoxetina,Capsula,Fluoxetina 20 mg,mg,Fluoxetina,Genfar,329.295281,Medio
4,Proximetacaina,Frasco,Proximetacaina 5 mg,ml,Alcaine,Alcon,64184.745760,Medio


In [5]:
df = df.drop(columns=['numerofactor'], errors='ignore')
df.shape

(10000, 8)

In [8]:
df.to_csv('medicamentos_limpio.csv', index=False)

Se seleccionaron ocho columnas relevantes del conjunto de datos, eliminando la variable numerofactor por no aportar información de gran relevancia para el análisis. El conjunto final se exportó en formato CSV para su posterior análisis y limpieza como se indica en el documento de lineamientos.